A demonstraction of vectorized "integration along a ray". Highly simplified to support adapation for the purposes of computing atmospheric phase delay. Missing some important steps partly because I am not clear on all parts of the delay model.

Vectorizing has some obvious drawbacks:

1. less intuitive interface (passing large vector arrays rather than points)
2. Larger memory footprint (store all ray vectors in memory)

The clear benefit though are:

1. use scipy/scipy/numpy directly without having to maintain external cython code
2. Straightforward (I think) to use dask

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import numpy as np
from tqdm import tqdm

# Context

We are going to look at ray tracing within the unit cube. We want to integrate at every point in space. The integral should be the length of the vector which intersects the unit cube. Thus we can compute it in two different ways:

1. the euclidean formula
2. integration in xarray

The second method will be much faster given it has analytic formulas namely (find the end point and take its norm). This provides a test to ensure correctness. Both methods (assuming our integration step is small enough) should be close (modulo some numerical approximation errors which is beyond the scope of this notebook).

## Parameters

Let us consider the unit cube $[0, 1]^3$ in $x \times y \times z$. We are going to have the spacing be $N \times N \times N_z$. Below are the parameters for this spacing.

In [2]:
N = 1000
N_z = 10

Do we want to do basic sanity checks? Answer: no if you are using $N \geq 50$

In [3]:
CHECKS = False

We also need:
    
- how many points per ray
- spacing along ray to integrate
- constant direcion.

In [4]:
STEP_SIZE = .1
RAY_DIRECTION = [1, 1, 1]  # (d_x , d_y, d_z)

Currently, we can get the minimum number of points to go alont the super-diagonal (all other rays will be shorter).

In [5]:
N_POINTS = int(np.ceil(STEP_SIZE**-1 * np.sqrt(3))) + 1
N_POINTS

19

# Basic checks

In [6]:
unit_ray = np.array(RAY_DIRECTION) / np.linalg.norm(RAY_DIRECTION)
r_x, r_y, r_z = unit_ray 
r_x, r_y, r_z

(0.5773502691896258, 0.5773502691896258, 0.5773502691896258)

Check the step size goes beyond the unit cube at its longest point.

In [7]:
np.all(N_POINTS * STEP_SIZE * unit_ray > np.array([1, 1, 1]))

True

We are going to construct the ray beyond the unit cube if required based on the direction.

# Dataset Initialization

## Coordinates

In [8]:
x_coord = np.linspace(0, 1, N)
y_coord = np.linspace(0, 1, N)
z_coord = np.linspace(0, 1, N_z)

Important note: for atmospheric ray tracing, we will need to let $z$ be the DEM Ellipsoidal height (I believe).\

## The data to be integrated

Just the 1-cube

In [9]:
%%time

D = np.ones((N, N, N_z))
D.shape

CPU times: user 14.1 ms, sys: 16.2 ms, total: 30.3 ms
Wall time: 28.6 ms


(1000, 1000, 10)

In [10]:
dims=['x', 'y' , 'z']
coords_dict = {'x': x_coord, 'y': y_coord, 'z': z_coord}

In [11]:
data_cube = xr.DataArray(data=D,
                         dims=dims,
                         coords=coords_dict)

## Setting up the ray

Within xarray.

In [12]:
d_x = xr.DataArray(data=r_x * np.ones((N, N, N_z)),
                   dims=dims,
                   coords=coords_dict)
d_y = xr.DataArray(data=r_y * np.ones((N, N, N_z)),
                   dims=dims,
                   coords=coords_dict)
d_z = xr.DataArray(data=r_z * np.ones((N, N, N_z)),
                   dims=dims,
                   coords=coords_dict)

Let's also initialize some datarrays for our computation.

In [13]:
ray_length_analytic = xr.DataArray(data=np.zeros((N, N, N_z)),
                          dims=dims,
                          coords=coords_dict)
ray_length_integration_loop = xr.DataArray(data=np.zeros((N, N, N_z)),
                                           dims=dims,
                                           coords=coords_dict)

## The dataset

In [14]:
ds = xr.Dataset({'data_cube':data_cube,
                 'd_x': d_x,
                 'd_y': d_y,
                 'd_z': d_z,
                 'ray_length_analytic': ray_length_analytic,
                 'ray_length_integration_loop': ray_length_integration_loop
                })
ds

<xarray.Dataset>
Dimensions:                      (x: 1000, y: 1000, z: 10)
Coordinates:
  * x                            (x) float64 0.0 0.001001 0.002002 ... 0.999 1.0
  * y                            (y) float64 0.0 0.001001 0.002002 ... 0.999 1.0
  * z                            (z) float64 0.0 0.1111 0.2222 ... 0.8889 1.0
Data variables:
    data_cube                    (x, y, z) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    d_x                          (x, y, z) float64 0.5774 0.5774 ... 0.5774
    d_y                          (x, y, z) float64 0.5774 0.5774 ... 0.5774
    d_z                          (x, y, z) float64 0.5774 0.5774 ... 0.5774
    ray_length_analytic          (x, y, z) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ray_length_integration_loop  (x, y, z) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

# Simplified Approach (with loops)

## Compution using Euclidean Norm

Let's use a nonvectorized form of the ray creation and compute length. We are also going to interpolate per point which will also be slower. This is for demonstration.

In [15]:
def get_ray(p_x: float, 
            p_y: float, 
            p_z: float,
            d_x: float, 
            d_y: float, 
            d_z: float,
            n_points=N_POINTS,
            step_size=STEP_SIZE):

    ray_start = np.array([p_x, p_y, p_z])
    ray_direction = np.array([d_x, d_y, d_z])

    ray_init = np.repeat(np.arange(n_points)[:,None],
                         3, 
                         axis=1)
    ray = ray_start + step_size * ray_init * ray_direction
    return ray

def compute_length(p_x: float,
                   p_y: float,
                   p_z: float, 
                   d_x: float, 
                   d_y: float, 
                   d_z: float) -> float:
    """
    In the unit cube!
    """
    
    ray = get_ray(p_x, p_y, p_z, d_x, d_y, d_z)
    # source: https://stackoverflow.com/a/432289
    # want to get intersection with unit-cube
    itemindex = np.where(ray>=1)
    ray_endpoint = ray[itemindex[0][0]]
    return np.linalg.norm(ray_endpoint - ray[0, ...])

def compute_length_integration(p_x: float, p_y: float, p_z: float) ->float:
    d_x = ds['d_x'].sel(x=p_x, y=p_y, z=p_z).data
    d_y = ds['d_y'].sel(x=p_x, y=p_y, z=p_z).data
    d_z = ds['d_z'].sel(x=p_x, y=p_y, z=p_z).data

    ray = get_ray(p_x, p_y, p_z, d_x, d_y, d_z)
    ray_x, ray_y, ray_z = ray[...,0], ray[..., 1], ray[..., 2]

    ray_x = xr.DataArray(ray_x, dims="step")
    ray_y = xr.DataArray(ray_y, dims="step")
    ray_z = xr.DataArray(ray_z, dims="step")

    z = ds['data_cube'].interp(x=ray_x, y=ray_y, z=ray_z, kwargs={"fill_value": 0})
    return np.trapz(z.data, dx=STEP_SIZE)

What is `compute_length` doing? Basically this:

In [16]:
np.linalg.norm([(N_POINTS - 1) * STEP_SIZE * r_x, 
                (N_POINTS - 1) * STEP_SIZE * r_y, 
                (N_POINTS - 1) * STEP_SIZE * r_z])

1.8000000000000003

In [17]:
compute_length(0, 0, 0, r_x, r_y, r_z)

1.8000000000000003

This should be the length of the vecto [1, 1, 1]

In [18]:
np.linalg.norm([1, 1, 1])

1.7320508075688772

This gets a fair bit closer. This function is just a demonstration and will be used within the loops below.

In [19]:
compute_length_integration(0, 0, 0)

1.7500000000000002

### Loop with Computing Analytic Distance

Note because we have to use `np.where` the values computed using this formula will be slightly greater than those computed with integrate since we are finding an *approximate* intersection with the unit cube.

In [20]:
if CHECKS:
    total = N**2 * N_z


    ### Notes:
    ## point in space p_x, p_y, p_z
    ## direction of ray d_x, d_y, d_z

    coord_gen = ((x, y, z) 
                   for x in ds.coords['x'].data 
                   for y in ds.coords['y'].data 
                   for z in ds.coords['z'].data)

    for xs in tqdm(coord_gen, total=total):
        p_x, p_y, p_z = xs
        d_x = ds['d_x'].sel(x=p_x, y=p_y, z=p_z).data
        d_y = ds['d_y'].sel(x=p_x, y=p_y, z=p_z).data
        d_z = ds['d_z'].sel(x=p_x, y=p_y, z=p_z).data

        ds['ray_length_analytic'].loc[dict(x=p_x,y=p_y,z=p_z)] = compute_length(p_x, p_y, p_z,
                                                                                d_x, d_y, d_z)
    

### Loop with Distance with Integration

In [21]:
if CHECKS:
    total = N**2 * N_z

    coord_gen = ((x, y, z) 
                   for x in ds.coords['x'].data 
                   for y in ds.coords['y'].data 
                   for z in ds.coords['z'].data)

    for xs in tqdm(coord_gen, total=total):
        p_x, p_y, p_z = xs
        d_x = ds['d_x'].sel(x=p_x, y=p_y, z=p_z).data
        d_y = ds['d_y'].sel(x=p_x, y=p_y, z=p_z).data
        d_z = ds['d_z'].sel(x=p_x, y=p_y, z=p_z).data

        ray = get_ray(p_x, p_y, p_z, d_x, d_y, d_z)
        ray_x, ray_y, ray_z = ray[...,0], ray[..., 1], ray[..., 2]

        ray_x = xr.DataArray(ray_x, dims="step")
        ray_y = xr.DataArray(ray_y, dims="step")
        ray_z = xr.DataArray(ray_z, dims="step")

        z = ds['data_cube'].interp(x=ray_x, y=ray_y, z=ray_z, kwargs={"fill_value": 0})    
        ds['ray_length_integration_loop'].loc[dict(x=p_x,y=p_y,z=p_z)] = np.trapz(z.data, dx=STEP_SIZE)


## Check our simplified first attempt

This simply takes the difference of the two methods. We expect the error (from both) to show up.

In [22]:
if CHECKS:
    plt.hist((ds['ray_length_integration_loop'].data - ds['ray_length_analytic'].data).ravel())
    plt.xlabel('Differences between two methods')

I am not going to investigate this further so hopefully this is a good enough sanity check.

There are several things that are going tobe slow in the above:

1. Per point assignment
2. Looping through points for ray creation, interpolation, and integration

Vectorizing tries to do this all at once creating fairly large arrays, but allowin scipy to multithread the lookups required to do things quickly.

# Vectorization

We are going to vectorize each part adding on to the previous modification and make sure they agree with our initial attempt. Namely:

1. ray creation
2. Interpolation
3. Integration

## Ray creation

Let $M = N^2 \cdot N_z$. The goal is to create an array of the following dimension $N_{steps} \times M \times 3$.

In [23]:
coord_mesh = np.meshgrid(x_coord, y_coord, z_coord)
coords_arr = np.stack(coord_mesh, axis=0).reshape((3, -1)).transpose([1, 0])
coords_arr.shape

(10000000, 3)

In [24]:
ind_mesh = np.meshgrid(np.arange(N), np.arange(N), np.arange(N_z))
indices_arr = np.stack(ind_mesh, axis=0).reshape((3, -1)).transpose([1, 0])
indices_arr.shape

(10000000, 3)

In [25]:
%%time

ray_init = np.repeat(np.arange(N_POINTS)[:,None],3, axis=1).transpose([1, 0])
ray_init = np.repeat(ray_init[None, ...], indices_arr.shape[0], axis=0).transpose([2, 0, 1])
ray_init.shape

CPU times: user 709 ms, sys: 852 ms, total: 1.56 s
Wall time: 1.55 s


(19, 10000000, 3)

In [26]:
%%time

ray = coords_arr + STEP_SIZE * ray_init * unit_ray
ray.shape

CPU times: user 4.39 s, sys: 3.39 s, total: 7.78 s
Wall time: 7.78 s


(19, 10000000, 3)

In [27]:
%%time

ray_dim_x, ray_dim_y, ray_dim_z = ray.shape
ray_squeezed = ray.reshape((ray_dim_x *ray_dim_y, ray_dim_z))

CPU times: user 5.56 s, sys: 1.05 s, total: 6.61 s
Wall time: 6.61 s


# Interpolation

In [28]:
from scipy.interpolate import RegularGridInterpolator
interp = RegularGridInterpolator((ds.coords['x'].data,
                                  ds.coords['y'].data,
                                  ds.coords['z'].data),
                                  ds['data_cube'].data, bounds_error=False, fill_value=0)

In [29]:
%%time

out = interp((ray_squeezed[:,0], 
              ray_squeezed[:,1], 
              ray_squeezed[:,2]))

CPU times: user 1min 9s, sys: 33.7 s, total: 1min 43s
Wall time: 1min 43s


In [30]:
%%time

out = out.reshape((ray_dim_x, ray_dim_y))
out.shape

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 19.8 µs


(19, 10000000)

# Integration

In [31]:
z = np.trapz(out, dx=STEP_SIZE, axis=0)
z.shape

(10000000,)

In [32]:
ray_length_integration_vec = xr.DataArray(data=z.reshape((N, N, N_z)),
                                          dims=dims,
                                          coords=coords_dict)
ds['ray_length_integration_vec'] = ray_length_integration_vec

In [33]:
if CHECKS:
    plt.hist((ds['ray_length_integration_loop'].data - ds['ray_length_integration_vec'].data).ravel())
    plt.xlabel('Differences between two methods')

# Concluding Thoughts

The memory issue quickly becomes problematic because we are storing these large rays in memory *simultaneously*. Dask could provide some reprieve however we would need to make sure that dask could access the data cube(s) while chunking the rays (likely copying the data cube or passing the scipy interpolation object). This seems easily doable with what is done above: https://examples.dask.org/xarray.html#Custom-workflows-and-automatic-parallelization

Vectorization is huge speedup. Looking at the cells, I am seeing about ~20 times, though it could be a lot more. I was able to do the ray integration of $N=1,000$ and `STEP_SIZE=.1` (20 points on the ray) in a few minutes.

When I kicked up $N = 10,000$ and `STEP_SIZE =0.01`, I ran out of memory. I was too impatient for $N = 1,000$ and the same step size (definitely petering on the edge of memory availability).

Although the unit ray vector here represents a simplified unit vector varying along the topography, the vectorization of the routines would remain very similar. 